<a href="https://colab.research.google.com/github/arssite/GENAi-Assessment/blob/main/ATS_Resume_Using_Bart_Facebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install gradio transformers python-docx spacy torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 469.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2

In [4]:
import gradio as gr
from transformers import pipeline
import spacy
from docx import Document
import tempfile
import os
import torch

# Load spaCy model
try:
    nlp = spacy.load('en_core_web_sm')
except IOError:
    import subprocess
    subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"])
    nlp = spacy.load('en_core_web_sm')

# Check for GPU availability
device = 0 if torch.cuda.is_available() else -1

# Load BART model for text generation
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)  # Use GPU if available

# Define functions
def extract_text_from_docx(file_path):
    try:
        doc = Document(file_path)
        return '\n'.join([para.text for para in doc.paragraphs])
    except:
        return ""

def save_updated_resume(text):
    try:
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.docx')
        doc = Document()
        for line in text.split('\n'):
            doc.add_paragraph(line)
        doc.save(temp_file.name)
        return temp_file.name
    except:
        return None

def bart_optimize_text(text):
    # Use BART to generate an optimized version of the text
    try:
        optimized_text = summarizer(text, max_length=300, min_length=100, do_sample=False)[0]['summary_text']
        return optimized_text
    except:
        return text  # Return original text if an error occurs

def process_resume(file):
    if file is None:
        return "Please upload a resume file.", "No suggestions yet.", None

    try:
        resume_text = extract_text_from_docx(file.name)
        if not resume_text:
            return "Could not read the resume. Please ensure it's a valid .docx file.", "No suggestions available.", None

        updated_text = bart_optimize_text(resume_text)  # Use BART for text optimization
        output_file_path = save_updated_resume(updated_text)

        return updated_text, "Text optimized using BART.", output_file_path
    except:
        return "An error occurred. Please try again with a different file.", "No suggestions available.", None

# Create and launch Gradio interface
iface = gr.Interface(
    fn=process_resume,
    inputs=gr.File(label="Upload Your Resume (.docx only)", file_types=[".docx"]),
    outputs=[
        gr.Textbox(label="Optimized Resume Text", lines=10),
        gr.Textbox(label="ATS Improvement Suggestions", lines=10),
        gr.File(label="Download Optimized Resume")
    ],
    title="ATS Resume Optimizer",
    description="Upload your .docx resume for ATS optimization. We'll suggest improvements and provide an updated version.",
    theme="default",
    css="""
        .gradio-container {max-width: 800px; margin: auto;}
        .output-markdown {white-space: pre-wrap;}
    """
)

iface.launch(share=True)


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1e17801fc1b72b38b8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
